In [ ]:
from App import App
import cv2
import numpy as np

app = App()

In [ ]:
image1 = cv2.imread('data_test/per1_1.jpg')
# object_images, results = app.detect_objects(image1)
results = app.yolo_model(image1)

In [ ]:
boxes = results[0].boxes

In [ ]:
print(boxes[0].xywh[0][1].item())

In [ ]:
import cv2
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms

class SiameseNetwork(nn.Module):
    def __init__(self, base_model):
        super(SiameseNetwork, self).__init__()
        self.base_model = base_model

    def forward(self, x1, x2):
        # Forward pass for both images
        out1 = self.base_model(x1)
        out2 = self.base_model(x2)
        return out1, out2

# # Create a base model (ResNet-18 in this case)
# base_model = models.resnet18(pretrained=True)
# # Modify the final fully connected layer to output a suitable dimension
# base_model.fc = nn.Linear(base_model.fc.in_features, 256)

# # Create a Siamese Network
# siamese_net = SiameseNetwork(base_model)

# Create a base model (ResNet-18 in this case)
base_model = models.resnet18(pretrained=True)
base_model.fc = nn.Linear(base_model.fc.in_features, 256)

# Quantize the model
quantized_model = torch.quantization.quantize_dynamic(
    base_model, {nn.Linear}, dtype=torch.qint8
)

# Create a Siamese Network with the quantized model
siamese_net = SiameseNetwork(quantized_model)

# Example forward pass with two images
# image1 = torch.randn(1, 3, 224, 224)  # replace with your image dimensions
# image2 = torch.randn(1, 3, 224, 224)  # replace with your image dimensions

# Example usage with OpenCV images
image_path1 = "data_test/test_reid/tower0_0.jpg"
image_path2 = "data_test/test_reid/tower1_0.jpg"

# Read images using OpenCV
image1 = cv2.imread(image_path1)
image2 = cv2.imread(image_path2)

# Check if images are read successfully
if image1 is None or image2 is None:
    print("Error: One or both images could not be read.")
else:
    # Convert OpenCV images to PyTorch tensors without PIL Image
    def preprocess_image(image):
        # Convert BGR to RGB
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Normalize to the range [0, 1]
        image_tensor = torch.from_numpy(image_rgb / 255.0).permute(2, 0, 1).float()
        # Add batch dimension
        image_tensor = image_tensor.unsqueeze(0)
        return image_tensor

    # Preprocess images
    input_tensor1 = preprocess_image(image1)
    input_tensor2 = preprocess_image(image2)

    # Example forward pass with two images
    embedding1, embedding2 = siamese_net(input_tensor1, input_tensor2)

    # Compute the similarity score (cosine similarity in this case)
    similarity_score = nn.functional.cosine_similarity(embedding1, embedding2)
    print(f"Similarity Score: {similarity_score.item()}")
    # Resnet50 đang là 43.6s
    # Resnet18 đang là 17.5s
    # Sử dụng thêm Quantom đang là 11.0s




In [ ]:
import cv2
import numpy as np

def color_histogram_similarity(image1, image2):
    hist1 = cv2.calcHist([image1], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist2 = cv2.calcHist([image2], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])

    # Normalize histograms
    hist1 = cv2.normalize(hist1, hist1).flatten()
    hist2 = cv2.normalize(hist2, hist2).flatten()

    # Clip values to ensure they are within valid range
    hist1 = np.clip(hist1, 0, 1)
    hist2 = np.clip(hist2, 0, 1)
    import matplotlib.pyplot as plt
    # Display histograms
    plt.plot(hist1, color='red', label='Image 1')
    plt.plot(hist2, color='blue', label='Image 2')
    plt.legend()
    plt.show()
    # Calculate histogram intersection similarity
    similarity = cv2.compareHist(hist1, hist2, cv2.HISTCMP_INTERSECT)
    return similarity


# Example usage
image_path1 = "data_test/test_reid/tower0_0.jpg"
image_path2 = "data_test/test_reid/tower1_0.jpg"

image1 = cv2.imread(image_path1)
image2 = cv2.imread(image_path2)

# Resize images for consistency (optional)
image1 = cv2.resize(image1, (224, 224))
image2 = cv2.resize(image2, (224, 224))

# Calculate color histogram similarity
similarity_score = color_histogram_similarity(image1, image2)
print(f"Color Histogram Similarity Score: {similarity_score}")


In [ ]:
import cv2
import numpy as np

def are_images_equal(image1, image2):
    # Đọc ảnh
    img1 = cv2.imread(image1)
    img2 = cv2.imread(image2)

    # Chuyển đổi ảnh sang không gian màu HSV
    hsv1 = cv2.cvtColor(img1, cv2.COLOR_BGR2HSV)
    hsv2 = cv2.cvtColor(img2, cv2.COLOR_BGR2HSV)

    # Tính histogram của ảnh
    hist1 = cv2.calcHist([hsv1], [0, 1], None, [180, 256], [0, 180, 0, 256])
    hist2 = cv2.calcHist([hsv2], [0, 1], None, [180, 256], [0, 180, 0, 256])

    # Chuẩn hóa histogram
    cv2.normalize(hist1, hist1, 0, 1, cv2.NORM_MINMAX)
    cv2.normalize(hist2, hist2, 0, 1, cv2.NORM_MINMAX)

    # Tính độ tương đồng giữa hai histogram
    similarity = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)

    # Xác định ngưỡng để quyết định hai ảnh có chụp cùng đối tượng hay không
    threshold = 0.8  # Ngưỡng có thể điều chỉnh

    if similarity > threshold:
        return True
    else:
        return False

# Đường dẫn của hai ảnh cần so sánh
image_path1 = "data_test/test_reid/tower0_0.jpg"
image_path2 = "data_test/test_reid/tower1_0.jpg"

# Kiểm tra xem hai ảnh có chụp cùng đối tượng hay không
result = are_images_equal(image_path1, image_path2)

if result:
    print("Hai ảnh chụp cùng đối tượng.")
else:
    print("Hai ảnh không chụp cùng đối tượng.")


In [ ]:
cam1 = '/home/harito/Videos/Cam1.mp4'
cam2 = '/home/harito/Videos/Cam2.mp4'

cap1 = cv2.VideoCapture(cam1)
cap2 = cv2.VideoCapture(cam2)

In [ ]:
ret1, frame1 = cap1.read()
ret2, frame2 = cap2.read()
real_frame = cv2.hconcat([frame1, frame2])

frame1_detect = app.detect_objects(frame1)
frame2_detect = app.detect_objects(frame2)
frame1_detect = cv2.cvtColor(np.array(app.draw_picture_detect(frame1_detect[1])), cv2.COLOR_RGB2BGR)
frame2_detect = cv2.cvtColor(np.array(app.draw_picture_detect(frame2_detect[1])), cv2.COLOR_RGB2BGR)

detect_frame = cv2.hconcat([frame1_detect, frame2_detect])

In [ ]:
print(real_frame.shape)
print(detect_frame.shape)

In [ ]:
frame1_detect = app.detect_objects(frame1)

In [ ]:
print(frame1_detect[1])

In [ ]:
detected_objects = []
# for box in frame1_detect[1][0].boxes:
#     detected_objects.append(box.cls())
# print(detected_objects)
print(frame1_detect[1][0].boxes)

In [ ]:
# print(frame1_detect[1][0].boxes[0])
type(frame1_detect[1][0].boxes[0].cls.item())

In [ ]:
import pickle

face_db_regis = pickle.loads(open("database/face_dictionary.pkl", "rb").read())

In [ ]:
print(type(face_db_regis))
print(type(face_db_regis['encodings']))
print(face_db_regis['encodings'])

In [1]:
from Main import Main
main = Main()
print(main.reid_dict)

Setting YOLO model for detect ...
Set reid with histogram
Setting the dictionary {name: vector feature} from load_data_base() ...
Set face recognition model with face_recognition 
Setup done!
{'Duc': <Main.List object at 0x7ff30bbdca50>, 'DucAnh': <Main.List object at 0x7ff318ba53d0>, 'Hai': <Main.List object at 0x7ff30d929150>, 'Thang': <Main.List object at 0x7ff30bbc1d10>}


In [3]:
print(main.reid_dict['Duc'].data[0])

[[    0.32005  0.00028474           0 ...           0           0    0.001139]
 [          0           0           0 ...           0           0   0.0014237]
 [          0           0           0 ...           0           0   0.0019932]
 ...
 [          0           0           0 ...           0           0           0]
 [          0           0           0 ...           0           0           0]
 [          0           0           0 ...           0           0           0]]
